In [ ]:
#!pip install webdriver-manager
#!pip install --upgrade selenium webdriver-manager
from selenium import webdriver
from openpyxl import Workbook
# from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import os
import requests
import base64
from selenium.webdriver.chrome.options import Options

def convert_image_to_binary(image_url):
    try:
        # Send a GET request to the image URL
        response = requests.get(image_url)
        # Raise an exception if the request was unsuccessful
        response.raise_for_status()
        
        # Convert the response content to a base64 encoded string
        image_binary_str = base64.b64encode(response.content).decode('utf-8')
        return image_binary_str
    except requests.RequestException as e:
        # Handle any errors that occur during the request
        print(f"An error occurred while fetching the image: {e}")
        return None
    
def safe_click(driver, element):
    try:
        element.click()
    except ElementClickInterceptedException:
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.click()

# Set up the Selenium WebDriver using the ChromeDriverManager which automatically manages the driver executable.
# service = Service(executable_path=ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service)

# Go to the desired website
driver=webdriver.Edge()
driver.get('https://staging.tucker-bloom.com/web/login')

# Log in if necessary
# Wait and fill the email and password
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'login'))).send_keys('gmail.com')
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'password'))).send_keys('yourpassword')

# Click the login button
login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))  # Replace with the correct selector for the login button
login_button.click()


try:
    inventory_link_xpath = "//a[@data-menu-id='108' and @data-action-id='231']"
    inventory_link = WebDriverWait(driver, 50).until(
        EC.visibility_of_element_located((By.XPATH, inventory_link_xpath))
    )
    inventory_link.click()
except TimeoutException:
    print("The element is not available after 20 seconds.")
    # Handle the exception, for example, by logging it or taking alternative actions

# Wait and click on the "Products" button to reveal the dropdown
products_button = WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.XPATH, "//span[@data-section='120']"))
)
products_button.click()

# After clicking the "Products" button, the dropdown appears
# Now click on the "Products" link within the dropdown menu
products_dropdown_link = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='#menu_id=124&action=214']"))
)
products_dropdown_link.click()

time.sleep(5) 

# Initialize the Excel workbook
workbook = Workbook()
sheet = workbook.active
sheet.append(['Product Name', 'Image Name', 'Image URL', 'Image in Binary'])

# Initialize the Selenium WebDriver and navigate to the product list page

# Start the loop to go through all pages
while True:
    # Wait until the products are loaded and get all products on the current page
    products = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".oe_kanban_global_click")))

    # Store the current URL to return after viewing a product
    current_page_url = driver.current_url
    size=len(products)
    index=0
    for index in range(size):
        # Re-locate the products to avoid stale element references
        while True:
            products = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".oe_kanban_global_click")))
            if(len(products)==size):
                break
        product = products[index]
        
        # Click the product to view its details
        safe_click(driver, product)
        #WebDriverWait(driver, 30).until(EC.element_to_be_clickable(product)).click()

        # Get the product name
        product_name_element = WebDriverWait(driver, 3600).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span.o_field_char.o_field_widget.o_quick_editable[name="name"]')))
        product_name = product_name_element.text
        print(product_name)

        # Try to click on the Images tab
        try:
            images_tab = WebDriverWait(driver, 3600).until(EC.element_to_be_clickable((By.LINK_TEXT, "Images")))
            images_tab.click()

            # Check if there are any images
            images = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.o_kanban_view.o_kanban_ungrouped .o_kanban_record:not(.o_kanban_ghost) img')))
            
            if not images:
                sheet.append([product_name, 'No images found', 'No URL found', 'No binary data'])
            else:
                for image in images:
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(image)).click()
                    image_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.o_field_char.o_field_widget.o_quick_editable[name="name"]'))).text
                    print(image_name)
                    image_url = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'img.img-fluid'))).get_attribute('src')
                    print(image_url)
                    image_binary_str = convert_image_to_binary(image_url)
                    print(image_binary_str)
                    sheet.append([product_name, image_name, image_url, image_binary_str])
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.close[data-dismiss="modal"]'))).click()
                    WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, '.modal-dialog')))

        except TimeoutException:
            sheet.append([product_name, 'No images found', 'No URL found', 'No binary data'])

        driver.back()
        WebDriverWait(driver, 30).until(EC.url_to_be(current_page_url))
        WebDriverWait(driver, 3600).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".oe_kanban_global_click")))
        index=index+1

    try:
        next_button = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.fa-chevron-right.btn.btn-secondary.o_pager_next")))
        if next_button.get_attribute("disabled"):
            break
        next_button.click()
        WebDriverWait(driver, 30).until(lambda d: d.current_url != current_page_url)
    except TimeoutException:
        break


# Save the Excel workbook
workbook.save('products_images.xlsx')

# Close the browser
driver.quit()
